In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.support import expected_conditions as EC
import os
import urllib ## lets us download images

import pickle as pk
import genanki
import time

Basically, we need a list of all of the bird codes and names. Then, we just plug in the media search url with the right taxonCode query:

https://media.ebird.org/catalog?searchField=region&regionCode=AU&mediaType=photo&sort=rating_rank_desc&taxonCode=**rebfir1**

## Get list of bird names and taxonomy ids..

In [ ]:
profile = webdriver.FirefoxOptions()
profile.set_preference("browser.download.folderList", 2)
# profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", "/home/ashley/Local Scripts/BIRDS/downloads")
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")

anu_account = "organisation"

URL = "https://ebird.org/region/AU?yr=all"

bird_dict = {}

with webdriver.Firefox(options=profile) as driver:

    driver.get(URL)

    # all_birds = driver.find_elements(By.TAG_NAME,"a")

    ## This is the class that contains all of the 'bird' boxes on this page. Need to inspect all of the elements under this class
    all_birds = driver.find_elements(By.CLASS_NAME,"Observation-species")

    # FUNCTION USED TO GET ALL ATTRBUTES OF AN ELEMENT!! VERY HELPFUL
    # print(driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', all_birds))


    ## Iterate through all of the elements with the appropriate class..
    for i in all_birds:

        ## Find the elements within this element that have the 'a' tag. This gives us the bird's taxonomy id
        a = i.find_elements(By.TAG_NAME,"a")

        ## likewise, the span class contains the name of the bird
        heading = i.find_elements(By.TAG_NAME,"span")

        ## Collect the attrbutes we want from these elements
        for j in a:
            tax_id = j.get_attribute("href")

        for j in heading:
            name = j.text

        ## If this is in fact the right element, save the corresponding bird names and ids into a dictionary
        if "https://ebird.org/species/" in tax_id:

            tax_id = tax_id.split("https://ebird.org/species/")[-1].split("/")[0]

            if "Exotic" not in name:
                bird_dict[tax_id] = name


pk.dump(bird_dict,open("bird_dict","wb"))

print(bird_dict)

## For every bird in the dictionary, collect information from ebird about ID and status

In [ ]:
bird_dict = pk.load(open("bird_dict","rb"))
profile = webdriver.FirefoxOptions()
profile.set_preference("browser.download.folderList", 2)
# profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", "/home/ashley/Local Scripts/BIRDS/downloads")
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")

with webdriver.Firefox(options=profile) as driver:
    URL = f"https://ebird.org/map/succoc"
    driver.get(URL)
    pwd = "C8l#mwwp$SdukWa6TnQrsxU#t"
    acc = "ashjbarnes"
    driver.find_element("id","input-user-name").send_keys(acc)
    driver.find_element("id","input-password").send_keys(pwd)
    driver.find_element("id","form-submit").click()


    for taxid in bird_dict:
        print(taxid,bird_dict[taxid])
        URL = f"https://ebird.org/species/{taxid}"
        driver.get(URL)
        element = driver.find_elements(By.ID,"conservation-status")

        ## Get Conservation Status
        for i in element:
            try:
                # print(i.find_element(By.CLASS_NAME,"Badge").text)
                bird_dict[taxid]["status"] = i.find_element(By.CLASS_NAME,"Badge").text

            except:
                print("notfound")
                bird_dict[taxid]["status"] = "Unknown"

                pass


        element = driver.find_elements(By.CLASS_NAME,"u-stack-sm")

        ## Get Info Text
        
        temp = 0
        for i in element:
            try:
                # print(i.text)
                if temp == 3:
                    infotext = i.text
                    # print(infotext)
                    bird_dict[taxid]["infotext"] = infotext
                temp += 1

            except:
                print("notfound")
                bird_dict[taxid]["status"] = "Not found"
                pass


        ## GET THE RANGE MAP
        URL = f"https://ebird.org/map/{taxid}"
        driver.get(URL)

        search =  WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"search-city")))

        search.send_keys("Australia")
        time.sleep(1)
        WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"Suggest-suggestion-0"))).click()
        # search.send_keys(Keys.RETURN)

        negtoggle = WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"negToggle")))
        negtoggle.click()

        map = WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"map-main")))
        time.sleep(4)
        map.screenshot(f"/home/ashley/Local_Scripts/BIRDS/downloads/map_{taxid}.png")



In [ ]:
bird_dict = pk.load(open("bird_dict","rb"))

## Now that we have a list of bird names and taxonomies, search for them all on the media page. This could be slow 

In [33]:
profile = webdriver.FirefoxOptions()
profile.set_preference("browser.download.folderList", 2)
# profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", "/home/ashley/Local Scripts/BIRDS/downloads")
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")
# profile.add_argument("--headless")

path = "/home/ashley/Local_Scripts/BIRDS/downloads/"

def download_image(url,name,path):

    ## Fix spaces in name
    tmp = name.split(" ")
    if len(tmp) > 1:
        name = tmp[0]
        for i in tmp[1:]:
            name += "\ " + i

    if url[-3:] == "480":
        url = url.split("480")[0] + "1200"
    os.system(f"wget -O {path}{name} {url} ")


def download_audio(url,name,path):

    ## Fix spaces in name
    tmp = name.split(" ")
    if len(tmp) > 1:
        name = tmp[0]
        for i in tmp[1:]:
            name += "\ " + i

    ## Replace "Poster" with "audio"

    url = url.split("poster")[0] + "audio"

    os.system(f"wget -O {path}{name} {url} ")


def fetch_media(bird_dict,path = "/home/ashley/Local_Scripts/BIRDS/downloads/"):

    with webdriver.Firefox(options=profile) as driver:
        ## Log into ebird to access map data
        URL = f"https://ebird.org/map/succoc"
        driver.get(URL)
        pwd = "C8l#mwwp$SdukWa6TnQrsxU#t"
        acc = "ashjbarnes"
        driver.find_element("id","input-user-name").send_keys(acc)
        driver.find_element("id","input-password").send_keys(pwd)
        driver.find_element("id","form-submit").click()

        downloaded = os.listdir("downloads")

        for taxid in bird_dict:
            name = bird_dict[taxid]

            if f"{taxid}.png" not in downloaded:
                print(taxid,bird_dict[taxid]["name"])

                URL = f"https://media.ebird.org/catalog?searchField=region&regionCode=AU&mediaType=photo&sort=rating_rank_desc&taxonCode={taxid}"


                driver.get(URL)

                img_objects = driver.find_element(By.TAG_NAME,"img")

                download_image(img_objects.get_attribute("src"),f"{taxid}.png",path)


                URL = f"https://media.ebird.org/catalog?searchField=region&regionCode=AU&mediaType=audio&sort=rating_rank_desc&taxonCode={taxid}"

                driver.get(URL)
                time.sleep(5)
                audio_objects = driver.find_element(By.CLASS_NAME,"ResultsGallery-image")
                href = audio_objects.get_attribute("src")
                download_audio(href,f"{taxid}.mp3",path)

                ## Get range map through some convoluted clicks and waits...

                URL = f"https://ebird.org/map/{taxid}"
                driver.get(URL)

                search =  WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"search-city")))

                search.send_keys("Australia")
                time.sleep(1)
                WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"Suggest-suggestion-0"))).click()
                # search.send_keys(Keys.RETURN)

                negtoggle = WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"negToggle")))
                negtoggle.click()

                map = WebDriverWait(driver,15).until(presence_of_element_located((By.ID,"map-main")))
                time.sleep(4)
                map.screenshot(f"{path}{taxid}_map.png")



    return


def fetch_info(bird_dict):
    with webdriver.Firefox(options=profile) as driver:

        for taxid in bird_dict:
            print(taxid,bird_dict[taxid]["name"])
            URL = f"https://ebird.org/species/{taxid}"
            driver.get(URL)
            element = driver.find_elements(By.ID,"conservation-status")

            ## Get Conservation Status
            for i in element:
                try:
                    # print(i.find_element(By.CLASS_NAME,"Badge").text)
                    bird_dict[taxid]["status"] = i.find_element(By.CLASS_NAME,"Badge").text

                except:
                    print("notfound")
                    bird_dict[taxid]["status"] = "Unknown"

                    pass


            element = driver.find_elements(By.CLASS_NAME,"u-stack-sm")

            ## Get Info Text
            
            temp = 0
            for i in element:
                try:
                    # print(i.text)
                    if temp == 3:
                        infotext = i.text
                        # print(infotext)
                        bird_dict[taxid]["infotext"] = infotext
                    temp += 1

                except:
                    print("notfound")
                    bird_dict[taxid]["status"] = "Not found"
                    pass

    return bird_dict



## Example of Use:

The following cell with create a downloads folder, then populate it with a sample of birds from a bird_dict

In [18]:
## Generate a small sample as a test case
bird_dict = pk.load(open("bird_dict","rb"))
bird_dict_sample = {}
i = 0
for bird in bird_dict:
    bird_dict_sample[bird] = bird_dict[bird]
    i += 1

    if i == 4:
        break



Now we have a sample bird dict, fetch the right info for each bird

In [26]:
bird_dict_sample = fetch_info(bird_dict_sample)

asikoe3 {'name': 'Pacific Koel'}
notfound
chbcuc2 {'name': 'Channel-billed Cuckoo'}
silgul2 {'name': 'Silver Gull'}
souboo8 {'name': 'Southern Boobook'}
notfound
litcor2 {'name': 'Little Corella'}
tawfro1 {'name': 'Tawny Frogmouth'}
auonig1 {'name': 'Australian Owlet-nightjar'}
maslap1 {'name': 'Masked Lapwing'}
notfound
dollar1 {'name': 'Dollarbird'}
angpet1 {'name': 'Southern Giant-Petrel'}


Download photos, maps and audio for each bird in the dictionary


In [32]:
fetch_media(bird_dict_sample,path = "/home/ashley/Local_Scripts/BIRDS/test")

asikoe3 {'name': 'Pacific Koel', 'status': 'Unknown', 'infotext': 'Large cuckoo known for its loud distinctive calls. Bright red eye. Males entirely black. Females barred with varying patterns, a black crown and a strong moustache. Occurs in a variety of habitats, fairly common in suburban areas. Strong migrant, spending the middle of the year in eastern Indonesia, New Guinea and the Solomon Islands, and migrating south into eastern Australia during the austral summer. Calls are very loud, often heard at night or pre-dawn, including a piercing ‘coo-eee’, and a longer ‘wirra-wirra-wirra’ (similar to a car alarm).'}
chbcuc2 {'name': 'Channel-billed Cuckoo', 'status': 'LC', 'infotext': 'Gigantic cuckoo, with a red eyering and enormous pale beak. A mostly pale gray bird, with darker wings and tail. Flies around in the early morning calling strongly, and typically roosts in fruiting fig trees. Lays its eggs in the nests of other species, especially Torresian Crow and Pied Currawong. Spends 

KeyboardInterrupt: 